### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    '''
    READ_ME:
    I changed this function on purpose, because previous produce concatenated words if there was no space
    after punctuation. I thought it could be better to keep all words sepate. That's why I get false in
    exercise 1a.
    '''
    return text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()

baby_df = pd.read_csv('/home/user/Pulpit/pani_ela_data_science/temat4/amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [84]:
# Bacause of the above reason, I'll do b) first ( than a) )
# b)
baby_df['review'] = baby_df['review'].fillna('')
baby_df["review"][38]

''

In [85]:
#a)
baby_df['review'] = baby_df['review'].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

False

In [86]:
baby_df["review"][4] # explained at the very beinning of the notebook

'All of my kids have cried non stop when I tried to ween them off their pacifier until I found Thumbuddy To Love s Binky Fairy Puppet It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from it This is a must buy book and a great gift for expecting parents You will save them soo many headaches Thanks for this book You all rock'

In [87]:
#b) - done before a

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [88]:
#c)
baby_df.drop(baby_df[baby_df.rating == 3].index, inplace=True)
#short test:
sum(baby_df["rating"] == 3)

0

In [89]:
#d) 
baby_df.rating.loc[baby_df['rating'] < 3] = -1
baby_df.rating.loc[(baby_df['rating'] == 4)] = 1
baby_df.rating.loc[(baby_df['rating'] == 5)] = 1

#short test:
sum(baby_df["rating"]**2 != 1)

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [90]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [91]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [92]:
#a)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(baby_df[['review']], baby_df[['rating']], test_size=0.33, random_state=42)

In [93]:
#b)
vectorizer = CountVectorizer()

X_train_vect = vectorizer.fit_transform(X_train.review)
X_test_vect = vectorizer.transform(X_test.review)
#print(X_train_vect.todense())

In [94]:
print(X_train_vect.shape)
print(y_train.shape)
print(X_test_vect.shape)

(111723, 52648)
(111723, 1)
(55029, 52648)


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [95]:
#a)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vect, y_train)

/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/user/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [96]:
#b)
# READ_ME:
# I assume that the entries with smallest coefficients are negative, and these with largest are positive
# Because I assume that we get a positive review if linear regression function is above threshold - wich 
# happens for numbers with large coeffibients. But what bothers me is that, for my argument to be true, I had 
# to assude that we have one linear regression curve (and assume positive to be these above threshold - but
# in fact its arbitrary,a nd I do not know whether it is the case). This situation is not the case for classi-
# fication example(from previous lecture) where we separately considered probability of being negative or pos.

most_probable_positives = []
most_probable_negatives = []
model_coefs = list(model.coef_[0])
for i in range(10):
    index_max = np.argmax(model_coefs)
    index_min = np.argmin(model_coefs)
    max_value = model_coefs.pop(index_max)
    min_value = model_coefs.pop(index_min)
    most_probable_positives.append( vectorizer.get_feature_names()[index_max] )
    most_probable_negatives.append( vectorizer.get_feature_names()[index_min] )

#vectorizer.get_feature_names()
#hint: model.coef_, vectorizer.get_feature_names()
#model.coef_[0]

In [97]:
print('Most probable positives:')
print(most_probable_positives)
print('Most probavle negatives:')
print(most_probable_negatives)

Most probable positives:
['wonderfully', 'con', 'mino', 'lifelong', 'reach', 'save', 'meetings', 'sonongram', 'perfectionalist', 'perfe']
Most probavle negatives:
['dissapointed', 'worships', 'worsens', 'worsened', 'worsen', 'worse', 'worrywart', 'worrying', 'worry', 'worrth']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [168]:
#a)
test_predictions = model.predict(X_test_vect)
test_predictions

array([-1, -1, -1, ...,  1,  1,  1])

In [169]:
#b)
test_probab = model.predict_proba(X_test_vect)
test_probab
#hint: model.predict_proba()
# The returned estimates for all classes are ordered by the label of classes, so in this case [P(-1),P(1)]

array([[0.58123368, 0.41876632],
       [0.84845375, 0.15154625],
       [0.97190261, 0.02809739],
       ...,
       [0.45390936, 0.54609064],
       [0.01561626, 0.98438374],
       [0.01050695, 0.98949305]])

In [170]:
#c) 
#len( np.sort(list(test_probab[:,0])) )

most_positive_rev_ind = []
most_negative_rev_ind = []
# (below I choosed revrsed order for positives and negatives for it to be compatible with a way of sorting
# performed by argsort)
positive_prob = list(test_probab[:,0])
negative_prob = list(test_probab[:,1])

positive_idx = np.argsort(positive_prob)[:5]
negative_idx = np.argsort(negative_prob)[:5]
print('Indices od most positive reviews:')
print(positive_idx)
print('Indices od most negative reviews:')
print(negative_idx)
print('')
print('Most positive reviews:')
print(X_test.review.iloc[positive_idx])
print('')
print('Most nagative reviews:')
print(X_test.review.iloc[negative_idx])

Indices od most positive reviews:
[ 4051  2724 36188  7283 46653]
Indices od most negative reviews:
[30933  4809 25937 53758   781]

Most positive reviews:
74899     We love this highchair We have a 4 year old an...
114796    My husband and I cannot state enough how much ...
22586     I researched a few different seats to put in o...
143615    I was a little nervous about ordering this bab...
28530     OVERVIEW My daughter has to be one of the wild...
Name: review, dtype: object

Most nagative reviews:
147902    My disappointment with this product prompted m...
175191    I had to return this stroller for three reason...
57234     My husband and I are VERY disappointed and sho...
16042     We have not had ANY luck with Fisher Price pro...
77072     I thought it sounded great to have different t...
Name: review, dtype: object


In [175]:
#d) 
# test_predictions
y_original = y_test.rating.tolist()
correct_pred = np.sum(test_predictions == y_original)
num_y = len(y_original)
accuracy = correct_pred * 1./ num_y
print(accuracy)

0.9315451852659506


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
# Please, send me the required set of significant words, because it is not provided.

In [103]:
#a)


In [104]:
#b)


In [105]:
#c)

#hint: %time, %timeit